In [177]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('logistic.txt', header=None, names = ['Exam1', 'Exam2', 'Admitted'])
data.insert(0, 'Ones', 1)

cols = data.shape[1]

X = data.iloc[:,:cols-1]
Y = data.iloc[:,cols-1:cols]


X = np.matrix(X)
Y = np.matrix(Y)

X1 = X.copy()

X1 = (X1[:,1:] - X1.mean(axis=0)[:,1:])/X1.std(axis=0)[:,1:]

X[:,1:] = X1
X.shape, Y.shape

((100, 3), (100, 1))

In [178]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
Y_onehot = encoder.fit_transform(Y)
Y_onehot.shape

(100, 2)

In [179]:
def sigmoid(z):
    
    return 1/(1 + np.exp(-z))

In [180]:
def forward_prop(X, theta1, theta2):
    
    X = np.matrix(X)
    theta1 = np.matrix(theta1)
    theta2 = np.matrix(theta2)
    m = X.shape[0]
    
    a1 = X
    z2 = a1 * (theta1.T)
    a2 = np.insert(sigmoid(z2), 0, np.ones(m), axis=1)
    z3 = a2 * (theta2.T)
    hx = sigmoid(z3)
    
    return a1, z2, a2, z3, hx
    


In [181]:
def cost(params, X, Y, input_size, hidden_size, num_labels, learning_rate):
    
    X = np.matrix(X)
    Y = np.matrix(Y)
    m = X.shape[0]
    
    theta1 = np.matrix(np.reshape(params[:((input_size + 1) * hidden_size)], (hidden_size, input_size + 1)))
    theta2 = np.matrix(np.reshape(params[(input_size + 1) * hidden_size:], (num_labels, hidden_size + 1)))
    
    a1, z2, a2, z3, hx = forward_prop(X, theta1, theta2)
    
    J = 0
    
    for i in range(m):
        
        sum1 = -(Y[i,:]) * (np.log(hx[i,:]).T)
        sum2 = (1 - Y[i,:]) * (np.log(1 - hx[i,:])).T
        
        J += np.sum(sum1 - sum2)
        
    J = J/m
    
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J
           

In [182]:
input_size = 2
hidden_size = 20
num_labels = 2
learning_rate = 6

In [183]:
params = (np.random.random(size = hidden_size*(input_size + 1) + num_labels*(hidden_size + 1)) - 0.5)*0.25
cost(params, X, Y_onehot, input_size, hidden_size, num_labels, learning_rate), params.shape

(1.4455919675261564, (102,))

In [184]:
def sigmoid_gradient(z):
    
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [185]:
def backprop(params, input_size, hidden_size, num_labels, X, Y, learning_rate):
    X = np.matrix(X)
    Y = np.matrix(Y)
    m = X.shape[0]
    
    theta1 = np.matrix(np.reshape(params[:(input_size + 1)*hidden_size], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size*(input_size + 1):], (num_labels, (hidden_size +1))))
    
    a1, z2, a2, z3 ,hx = forward_prop(X, theta1, theta2)
    
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    
    J = 0
    for i in range(m):
        sum1 = np.multiply(-Y[i,:], np.log(hx[i,:]))
        sum2 = np.multiply((1-Y[i,:]), np.log(1 - hx[i,:]))
        
        J += np.sum(sum1 - sum2)
        
    J = J/m
    
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        hxt = hx[t,:]
        Yt  = Y[t,:]
        
        d3t = hxt - Yt
        
#        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((d3t * theta2[:,1:]), sigmoid_gradient(z2t))
        
        delta1 = delta1 + d2t.T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
        
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
        
        
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad
           

In [186]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, Y_onehot, learning_rate)
J, grad.shape

(1.4455919675261564, (102,))

In [172]:
from scipy.optimize import minimize
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, Y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin.fun

1.1596101472469078

In [173]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_prop(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1))


In [174]:
np.sum(np.fabs(Y - y_pred))

8.0